In [1]:
'''
This script converts diploid data into 2 types that could be used in PopNet
'''
import pandas as pd, numpy as np
from pathlib import Path

In [2]:
dir = Path('/d/data/diploid')
input = dir / 'data.tsv'
output_fixed = dir / 'fixed.tsv' #Fixes the reference column
output_haploid = dir / 'haploid.tsv' #Split into haploid format

In [10]:
#creates 'fixed.tsv'. Results are okay? 
def fix_ref(ref_series):
    #replaces each element in the REF column with the diploid version: T => T/T
    def f(x):
        return "{0}/{0}".format(x)
    return ref_series.apply(f)

data = pd.read_csv(input, sep = '\t', header = 0, index_col = [0,1], dtype = 'str')

data["REF"] = fix_ref(data["REF"])
data.to_csv(output_fixed, sep = '\t')

In [19]:
#creates 'haploid.tsv'. Did not get good results with this in the end.
def split_diploid(df):
    #splits each column into two. T/T = [T, T]. The columns are named ERRX -> [ERRX_1, ERRX_2]
    res = pd.DataFrame(index = df.index)
    res['REF'] = df['REF']
    for c in df.columns[1:]:
        res[[c + "_1", c + "_2"]] = df[c].str.split("/", expand=True)
    return res

data = pd.read_csv(input, sep = '\t', header = 0, index_col = [0,1], dtype = 'str') #reload the dataframe

haploid_df = split_diploid(data)
haploid_df.to_csv(output_haploid, sep = '\t')